# Configuration Access

<b>This notebook is not supposed to be used on its own.<b>

In [1]:
from exasol.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for the styles NB from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}

In [ ]:
def get_access_store_ui(root_dir: str = '.'):
    
    from enum import Enum
    from pathlib import Path
    import ipywidgets as widgets
    from IPython.display import Javascript, display, clear_output
    from exasol.secret_store import Secrets
    
    # Try to find the file name in the shared store.
    # Create a global variable only temporarily.
    %store -r
    if 'sb_store_file' in globals():
        global sb_store_file
        sb_store_file_ = sb_store_file
        del sb_store_file
    else:
        sb_store_file_ = 'dss_config.sqlite'

    ui_look = get_config_styles()

    header_lbl = widgets.Label(value='Configuration Store', style=ui_look.header_style, layout=ui_look.header_layout)
    file_lbl = widgets.Label(value='File Path', style=ui_look.label_style, layout=ui_look.label_layout)
    password_lbl = widgets.Label(value='Password', style=ui_look.label_style, layout=ui_look.label_layout)
    file_txt = widgets.Text(value=sb_store_file_, style=ui_look.input_style, layout=ui_look.input_layout)
    password_txt = widgets.Password(style=ui_look.input_style, layout=ui_look.input_layout)
    open_btn = widgets.Button(description='Open', style=ui_look.button_style, layout=ui_look.button_layout)

    # Prepare to display an error message
    notify_output = widgets.Output()
    display(notify_output)

    @notify_output.capture()
    def popup_message(message):
        clear_output()
        display(Javascript(f"alert('{message}')"))

    def open_or_create_config_store(btn):
        global sb_config, sb_store_file
        sb_store_file = file_txt.value
        try:
            sb_config = Secrets(Path(root_dir) / sb_store_file, password_txt.value)
            sb_config.connection()
        except:
            popup_message('Failed to open the store. Please check that the password is correct')
        else:
            open_btn.icon = 'check'
        finally:
            # Save the file in the shared store.
            %store sb_store_file
            del sb_store_file
    
    open_btn.on_click(open_or_create_config_store)

    group_items = [
        header_lbl,
        widgets.Box([file_lbl, file_txt], layout=ui_look.row_layout),
        widgets.Box([password_lbl, password_txt], layout=ui_look.row_layout)
    ]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), open_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui